# Funciones índices de sequía

En este _notebook_ se guardan las funciones que se utilizan en los ejercicios de sequías, de manera que se pueden importar directamente en cualquier otro _notebook_ sin necesidad de reescribirlas. 

Para cargar estas funciones en otro _notebook_, se utiliza una de las funciones llamadas mágicas en terminología _Python_:
```Python
%run funciones_sequias.ipynb
```
Esta línea sólo funciona si `funciones_sequias.ipynb` está en la misma carpeta que el _notebook_ en el que se quiere cargar. En caso contrario, hay que utilizar el paquete `os` para definir en qué carpeta se encuentra `funciones_sequias.ipynb`.

```Python
import os
os.chdir('ruta donde se encuentra `funciones_sequias.ipynb')
%run funciones_precipitacion.ipynb
```

__Índice__<br>

__[Evapotranspiración](#Evapotranspiración)__<br>

__[Índices de sequía](#Índices-de-sequía)__<br>

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set()
plt.style.use('dark_background')
#plt.style.use('seaborn-whitegrid')
%matplotlib inline

from datetime import datetime, timedelta
import scipy.stats as stats
import tqdm

### Evapotranspiración

In [9]:
# Cargar funciones
%run funciones_sol.ipynb

In [10]:
def etp_thornthwaite(T, lat):
    """Calcula la serie de evapotranspiración potencial mediante el método de
    Thornthwaite:
    
                ETP = 16 * Ld * (10 * T / I)**a [mm]
                
                I = sum(i);     i = (Ti / 5)**1.514
                a = 6.75e-7 * I***3 - 7.71e-5 * I**2 + 1.791e-2 * I + 0.49239
    
    Entradas:
    ---------
    T:       Series. Serie mensual de temperatura media [ºC]
    lat:     float. Latitud del punto de cálculo en grados sexagesimales
    
    Salidas:
    --------
    etp:     Series. Serie mensual de evapotranspiración potencial [mm]"""
    
    # Longitud media del día para cada mes como múltiplo de 12 h
    lat_rad = lat * np.pi / 180
    Ld = _monthly_mean_daylight_hours(latitude_radians=lat_rad, leap=False)
    Ld /= 12
    Ld = pd.Series(Ld, index=range(1, 13))

    # Calcular el índice de calor anual
    I = 0
    Tm = T.groupby(T.index.month).mean()
    for Ti in Tm:
        I += (Ti / 5)**1.514 # índice de calor mensual
        
    # Calcular parámetro 'a'
    a = 6.75e-7 * I**3 - 7.71e-5 * I**2 + 1.791e-2 * I + 0.49239
    
    # Calcular la serie de etp
    etp = pd.Series(index=T.index)
    for idx in etp.index:
        mes = idx.month
        etp[idx] = 16 * Ld[mes] * (10 * T[idx] / I)**a
        
    return etp

### Índices de sequía

In [44]:
def SPI(serie_pcp, verbose=False):
    """Calcular el 'standard precipitation index' (SPI) de una serie de
    precipitación
    
    Entradas:
    ---------
    serie_pcp: Series. Serie de precipitación
    verbose:   boolean. Si se muestran los coeficientes ajustados para la
               distribución gamma
    
    Salidas:
    --------
    SPIs:      Series. Serie de SPI
    """
    
    # ajustar la función de distribución gamma
    alpha, loc, beta = stats.gamma.fit(serie_pcp, floc=0)
    if verbose == True:
        print('alpha = {0:.3f}\tloc = {1:.3f}\tbeta = {2:.3f}'.format(alpha, loc,
                                                                      beta))
    
    # calcular el SPI para la serie
    SPIs = pd.Series(index=serie_pcp.index)
    for idx, pcp in zip(serie_pcp.index, serie_pcp):
        cdf = stats.gamma.cdf(pcp, alpha, loc, beta)
        SPIs[idx] = stats.norm.ppf(cdf)
        
    return SPIs

In [50]:
def plot_SPI(serie_spi, title):
    """Crea un diagrama de línea con la evolución temporal del SPI
    
    Entradas:
    ---------
    serie_spi: Series. Serie temporal de SPI
    title:     string. Título del gráfico
    
    Salidas:
    --------
    Gráfico de línea"""
    
    # Configuración
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.set(xlim=(serie_spi.index[0], serie_spi.index[-1]), ylim=(-3, 3))
    ax.set_title(title, fontsize=14)
    
    # Gráfico de línea del SPI
    ax.plot(serie_spi, color='gold', linewidth=1.2)
    
    # Fondo con la leyenda de cada rango de SPI
    ax.fill_between(serie_spi.index, -3, -2, color='indianred', alpha=0.4,
                    label='sequía extrema')
    ax.fill_between(serie_spi.index, -2, -1.5, color='indianred', alpha=0.3,
                    label='sequía severa')
    ax.fill_between(serie_spi.index, -1.5, -1, color='indianred', alpha=0.2,
                    label='sequía moderada')
    ax.fill_between(serie_spi.index, -1, 0, color='indianred', alpha=0.1,
                    label='sequía ligera')
    ax.fill_between(serie_spi.index, 0, 1, color='steelblue', alpha=0.1,
                    label='húmedo ligero')
    ax.fill_between(serie_spi.index, 1, 1.5, color='steelblue', alpha=0.2,
                    label='húmedo moderado')
    ax.fill_between(serie_spi.index, 1.5, 2, color='steelblue', alpha=0.3,
                    label='húmedo severo')
    ax.fill_between(serie_spi.index, 2, 3, color='steelblue', alpha=0.4,
                    label='húmedo extremo')
    
    # leyenda
    fig.legend(loc=[0.15, 0], ncol=4, fontsize=11);